# Loading most recent data from postgres table

## Installing needed packages and connections

In [54]:
# Install libraries
!pip install psycopg2-binary
!pip install requests
!pip install sqlalchemy --upgrade

## Load data and analytics

In [31]:
from api_query import api_query_call

df = api_query_call("SELECT * FROM uniswap_table")


,block_timestamp_time,transaction_to_address,transaction_txfrom_address,close_price,basecurrency_symbol,quotecurrency_symbol
0,2023-05-03 22:39:47,0x0000000000a84d1a9b0063a910315c7ffa9cd248,0x5e3d1448a0365c3b1cd08908d410bfccca449c45,1904.8533148165884,ETH,USDC
1,2023-05-03 22:39:47,0x0000000000a84d1a9b0063a910315c7ffa9cd248,0x5e3d1448a0365c3b1cd08908d410bfccca449c45,0.0005249748063127298,USDC,ETH
2,2023-05-03 22:35:35,0xa88800cd213da5ae406ce248380802bd53b47647,0x9108813f22637385228a1c621c1904bbbc50dc25,1888.0772403321316,ETH,USDC
3,2023-05-03 22:35:35,0xa88800cd213da5ae406ce248380802bd53b47647,0x9108813f22637385228a1c621c1904bbbc50dc25,0.0005296393487716053,USDC,ETH
4,2023-05-03 22:35:35,0xa88800cd213da5ae406ce248380802bd53b47647,0x9108813f22637385228a1c621c1904bbbc50dc25,0.000031954331131291345,HEX,ETH


In [32]:
# Trim dataframe
df = df[["block_timestamp_time","transaction_to_address","transaction_txfrom_address","close_price","basecurrency_symbol" \
        ,"quotecurrency_symbol"]]

df[0:10].head()

,block_timestamp_time,transaction_to_address,transaction_txfrom_address,close_price,basecurrency_symbol,quotecurrency_symbol
0,2023-05-03 22:39:47,0x0000000000a84d1a9b0063a910315c7ffa9cd248,0x5e3d1448a0365c3b1cd08908d410bfccca449c45,1904.8533148165884,ETH,USDC
1,2023-05-03 22:39:47,0x0000000000a84d1a9b0063a910315c7ffa9cd248,0x5e3d1448a0365c3b1cd08908d410bfccca449c45,0.0005249748063127298,USDC,ETH
2,2023-05-03 22:35:35,0xa88800cd213da5ae406ce248380802bd53b47647,0x9108813f22637385228a1c621c1904bbbc50dc25,1888.0772403321316,ETH,USDC
3,2023-05-03 22:35:35,0xa88800cd213da5ae406ce248380802bd53b47647,0x9108813f22637385228a1c621c1904bbbc50dc25,0.0005296393487716053,USDC,ETH
4,2023-05-03 22:35:35,0xa88800cd213da5ae406ce248380802bd53b47647,0x9108813f22637385228a1c621c1904bbbc50dc25,0.000031954331131291345,HEX,ETH


In [33]:
# Select only the rows where City is 'Chicago'
USDC_Transactions = df[df['basecurrency_symbol'] == 'USDC']
USDC_Transactions.head()

,block_timestamp_time,transaction_to_address,transaction_txfrom_address,close_price,basecurrency_symbol,quotecurrency_symbol
1,2023-05-03 22:39:47,0x0000000000a84d1a9b0063a910315c7ffa9cd248,0x5e3d1448a0365c3b1cd08908d410bfccca449c45,0.0005249748063127298,USDC,ETH
3,2023-05-03 22:35:35,0xa88800cd213da5ae406ce248380802bd53b47647,0x9108813f22637385228a1c621c1904bbbc50dc25,0.0005296393487716053,USDC,ETH
17,2023-05-03 17:44:47,0x24902aa0cf0000a08c0ea0b003b0c0bf600000e0,0xd7e1236c08731c3632519dcd1a581bfe6876a3b2,0.0005347831499075275,USDC,ETH
24,2023-05-03 13:22:59,0xdef1c0ded9bec7f1a1670819833240f027b25eff,0x7322ec21a36a5c413f10f43ecbebb7961f9c9734,0.000540588419559359,USDC,ETH
36,2023-05-03 03:32:47,0x24902aa0cf0000a08c0ea0b003b0c0bf600000e0,0xd7e1236c08731c3632519dcd1a581bfe6876a3b2,0.0005364205609019324,USDC,ETH


In [18]:
# We can see the highest trading wallets
counts = df[['transaction_to_address', 'transaction_txfrom_address']].stack().value_counts()

# Print the wallet with the highest transactions
print(counts[[0]])



0x24902aa0cf0000a08c0ea0b003b0c0bf600000e0    166
dtype: int64


In [25]:
# Group the rows by object and sum up the cost
subset = df.iloc[:, :6]
print(subset.head())

   tradeindex block_timestamp_time  block_height exchange_fullname protocol  \
0          15  2023-05-03 22:39:47      17183209           Uniswap  Uniswap   
1          15  2023-05-03 22:39:47      17183209           Uniswap  Uniswap   
2          11  2023-05-03 22:35:35      17183188           Uniswap  Uniswap   
3          11  2023-05-03 22:35:35      17183188           Uniswap  Uniswap   
4          10  2023-05-03 22:35:35      17183188           Uniswap  Uniswap   

  timeinterval_minute  
0 2023-05-03 22:39:00  
1 2023-05-03 22:39:00  
2 2023-05-03 22:35:00  
3 2023-05-03 22:35:00  
4 2023-05-03 22:35:00  


In [23]:
# Group the rows by object and sum up the cost
subset = df.iloc[:, 6:15]
print(subset.head())

# total_costs = df.groupby('basecurrency_symbol')['cost'].sum()a

           baseamount         quoteamount  trades               quoteprice  \
0   2597.662901948303   2610.064734550981       1       1904.8533148165884   
1   2610.064734550981   2597.662901948303       1    0.0005249748063127298   
2  251.06246686521044  250.03942847251892       1       1888.0772403321316   
3  250.03942847251892  251.06246686521044       1    0.0005296393487716053   
4   252.8014772247965  251.06246686521044       1  0.000031954331131291345   

             maximum_price            minimum_price               open_price  \
0       1904.8533148165884       1904.8533148165884       1904.8533148165884   
1    0.0005249748063127298    0.0005249748063127298    0.0005249748063127298   
2       1888.0772403321316       1888.0772403321316       1888.0772403321316   
3    0.0005296393487716053    0.0005296393487716053    0.0005296393487716053   
4  0.000031954331131291345  0.000031954331131291345  0.000031954331131291345   

               close_price basecurrency_symbol  
0

In [24]:
subset = df.iloc[:, 15:]
print(subset.head())

                         basecurrency_address quotecurrency_symbol  \
0                                           -                 USDC   
1  0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48                  ETH   
2                                           -                 USDC   
3  0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48                  ETH   
4  0x2b591e99afe9f32eaa6214f7b7629768c40eeb39                  ETH   

                        quotecurrency_address  \
0  0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48   
1                                           -   
2  0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48   
3                                           -   
4                                           -   

                                    transaction_hash  transaction_gas  \
0  0xc57ff802cab146c1bf6c311988f8c361506e054ab51c...           407492   
1  0xc57ff802cab146c1bf6c311988f8c361506e054ab51c...           407492   
2  0x7c1d4981c688196c7135ab13f6e13f7da3c95040f5e8...           41